## Derivations of EM steps for Mixture of Beta Binomial Distribution

- Beta-Binomial Distribution (BB): 
$$
\begin{align}
p(y^{(i)} | \theta) &= \int_{0}^{(1)}Bin(y^i|n,p) \cdot Beta(p|\alpha, \beta) dp \\
                &= {{n}\choose{y^{(i)}}}\frac{1}{B(\alpha,\beta)} \int_{0}^{1} p^{y^{(i)}+ \alpha -1} (1-p)^{n - y^{(i)} + \beta - 1} dp   \\
                &= {{n}\choose{y^{(i)}}}\frac{B(y^{(i)}+\alpha, n-y^{(i)}+\beta)}{B(\alpha, \beta)}
\end{align}
$$

- Mixture of Beta-Binomial Distribution (MBB), joint probablility, suppose there are K components, and let the variables be $\mathbf{\gamma} = [\gamma_{0}, \gamma_{1}, ..., \gamma_{k}, \gamma_{K}]$, $\gamma_{k} \text{~} Bin(\gamma_{k}|\pi_k)$:
$$
\begin{align}
p(y^{(i)}, \gamma^{(i)} | \theta, \pi) &= \prod_{k=1}^{K} p(y^{(i)}, \gamma_{k}^{(i)}| \theta_{k}, \pi_k)^{\gamma^{(i)}_k} \\
                                                      &= \prod_{k=1}^{K} \{p(y^{(i)} | \theta_{k}) \pi_k \}^{\gamma_k^{(i)}}
\end{align}
$$

- Log likelihood of the full data (MBB):
$$
\begin{align}
\log p(y^{(i)}, \gamma^{(i)} | \theta, \pi) &= \sum_{k=1}^{K} \bigg \{ \gamma^{(i)}_k \big(\log\pi_k + \log p(y^{(i)} | \theta_{k}) \big)
\bigg\} 
\end{align}
$$

## E step:

$$
\begin{align}
E_{\gamma^{(i)}_k \text{~} p(\pi_k|y^{(i)})}[\log p(y^{(i)}, \gamma^{(i)}] &= E\bigg[\sum_{k=1}^{K} \big \{ \gamma^{(i)}_k \big(\log\pi_k + \log p(y^{(i)} | \theta_k) \big)\big\} \bigg] \\
                      &= \sum_{k=1}^{K} \big \{ E[\gamma^{(i)}_k] \big(\log\pi_k + \log p(y^{(i)} | \theta_k) \big)\big\} \\
\end{align}
$$


As:
$$
\begin{align}
\bar{\gamma}_k^{(i)} = E(\gamma^{(i)}_k|y^{(i)}, \theta, \pi) &= p(\gamma_k = 1| y^{(i)}, \theta, \pi)   \text(- Expection of Bernoulli distribution)\\
                                       &= \frac{p(\gamma^{(i)}_k=1, y^{(i)}| \theta, \pi)}{\sum_{k=1}^{K}p(\gamma_k^{(i)}=1, y^{(i)}| \theta, \pi)} \\
                                       &= \frac{p(y^{(i)}|\gamma^{(i)}_k=1, \theta, \pi) \cdot p(\gamma^{(i)}_k=1|\pi)}{\sum_{k=1}^{K}p(y^{(i)}|\gamma^{(i)}_k=1, \theta, \pi) \cdot p(\gamma^{(i)}_k=1|\pi)} \\
                                       &= \frac{ {{n}\choose{y^{(i)}}} \frac{B(y^{(i)} + \alpha_k, n-y^{(i)} + \beta_k)}{B(\alpha_k, \beta_k)} \cdot \pi_k}{ \sum_{k=1}^{K}{{n}\choose{y^{(i)}}} \frac{B(y^{(i)} + \alpha_k, n-y^{(i)} + \beta_k)}{B(\alpha_k, \beta_k)} \cdot \pi_k} \\
                                       &= \frac{\frac{B(y^{(i)} + \alpha_k, n-y^{(i)} + \beta_k)}{B(\alpha_k, \beta_k)} \cdot \pi_k}{ \sum_{k=1}^{K} \frac{B(y^{(i)} + \alpha_k, n-y^{(i)} + \beta_k)}{B(\alpha_k, \beta_k)} \cdot \pi_k}
\end{align}
$$

## M step
M-step hence is to optimize:

$$
 \max_{\pi, \theta}\bigg\{\sum_{k=1}^{K} \big \{ \bar{\gamma}_k^{(i)} \big(\log\pi_k + \log p(y^{(i)} | \theta_k) \big)\big\} \bigg\} \\
 = \max_{\pi, \theta}\bigg\{ \sum_{k=1}^{K} \big \{ \bar{\gamma}_k^{(i)} \big(\log\pi_k + \log{{n}\choose{y^{(i)}}} + \log B(y^{(i)}+\alpha_k, n-y^{(i)}+\beta_k) - \log{B(\alpha_k, \beta_k)} \big)\big\} \bigg\} \\
s.t. \sum_{k=1}^{K} \pi_k = 1, \\ \pi_k \geq 0, k=1,...,K\\ \alpha > 0, \\ \beta > 0
$$


Using all the data points:

$$
\max_{\pi, \theta}\bigg\{ \sum_{i=1}^{N}\sum_{k=1}^{K} \big \{ \bar{\gamma}_k^{(i)} \big(\log\pi_k + \log{{n}\choose{y^{(i)}}} + \log B(y^{(i)}+\alpha_k, n-y^{(i)}+\beta_k) - \log{B(\alpha_k, \beta_k)} \big)\big\} \bigg\} \\
s.t. \sum_{k=1}^{K} \pi_k = 1 \\ \pi_k \geq 0, k=1,...,K \\ \alpha > 0, \\ \beta > 0
$$

In [26]:
import numpy as np
from scipy.special import beta, comb, gammaln
from scipy.optimize import minimize
from sklearn.cluster import KMeans
from scipy.stats import betabinom, bernoulli
import sys

In [113]:
class MixtureBetaBinomial:
    
    def __init__(self, 
                 n_components=2,
                 max_m_step_iter=250,
                 tor=1e-12
                ):
        super(MixtureBetaBinomial, self).__init__()
        self.n_components = n_components
        self.max_m_step_iter = max_m_step_iter
        self.tor = tor
        self.params = None
        
    def _E_step(self, data, a, b, pi):
        y, n = data
        log_E_gamma_k = np.log(pi) + gammaln(y+a) + gammaln(n-y+b) + gammaln(a+b) - \
                (gammaln(a) + gammaln(b) + gammaln(n+a+b))
        return np.exp(log_E_gamma_k)
    
    def _get_constraints(self):
        constraints = ({'type':'eq', 'fun': lambda params: np.sum(params[2*self.n_components:]) - 1})
        return constraints
    
    def _get_bounds(self):
        bounds = [(0, None)]* (3*self.n_components)
        return tuple(bounds)
    
    def _get_nnl_fun(self):
        def neg_log_likelihood(params, *args):
            """
            params: 1-D arrays: [a_0, a_1, a_2, ..., b_0, b_1, b_2, ..., pi_0, pi_1, pi_2,...]
            args: list of function arguments, 
                args[0] is the data: tuple of list (y, n)
                args[1] is \hat{\gamma}, expectation of the latent variables
                args[2] is number of components, int
            """
            y, n = args[0]
            E_gammas = args[1]
            n_components = args[2]

        #     log_likelihood = 0
        #     for i in range(n_components):
        #         a, b, pi = params[i], params[i+n_components], params[i+2*n_components]
        #         E_gamma_k = E_gammas[i]
        #         log_pdf =  E_gamma_k * (np.log(pi) + np.log(comb(n, y)) + np.log(beta(y + a, n - y + b)) - np.log(beta(a, b)))
        #         log_likelihood += np.sum(log_pdf)

            # numerical stabler computation when n and y are very large
            # https://en.wikipedia.org/wiki/Beta-binomial_distribution#Beta-binomial_distribution_as_a_compound_distribution
            log_likelihood = 0
            for i in range(n_components):
                a, b, pi = params[i], params[i+n_components], params[i+2*n_components]
                E_gamma_k = E_gammas[i]
                log_pdf = E_gamma_k * (np.log(pi) + gammaln(n+1) + gammaln(y+a) + gammaln(n-y+b) + gammaln(a+b) - \
                 (gammaln(y+1) + gammaln(n-y+1) + gammaln(a) + gammaln(b) + gammaln(n+a+b)))
                log_likelihood += np.sum(log_pdf)
            return -log_likelihood
        
        return neg_log_likelihood
    
    def _M_step(self, data, E_gammas, params):
        constraints = self._get_constraints()
        bounds = self._get_bounds()
        nll_fun = self._get_nnl_fun()
        res = minimize(nll_fun, x0=params,
                args=(data, E_gammas, self.n_components),
                bounds=bounds,
                method='SLSQP',
                options={'disp': True, 'maxiter': self.max_m_step_iter},
                constraints=constraints #  for solvers: COBYLA, SLSQP and trust-constr
                )
        return res
    
    def _perturbate(self, array):
        array += np.random.uniform(-0.025, 0.025, np.shape(array))
        while np.any(array) < 0 or np.any(array) > 1:
            array += np.random.uniform(-0.025, 0.025, np.shape(array))
        return array
        
    
    def _init_gamma_with_kmeans(self, data):
        # set all alpha and beta to be 0.5
        y, n = data
        sample_size = len(y)
        proportions = y / n
        kmeans = KMeans(
            n_clusters=self.n_components,
            max_iter=500,
            random_state=44
           )
        kmeans.fit(proportions.reshape(-1,1))
        gammars = np.zeros((sample_size, self.n_components), dtype=np.float) 
        gammars[range(sample_size), kmeans.labels_] = 1.0
        
        pi = np.sum(gammars, axis=0)
        pi = pi / np.sum(pi)
        
        params = np.concatenate([np.ones(self.n_components, dtype=np.float)*0.9, 
                                 np.ones(self.n_components, dtype=np.float)*0.9,
                                 pi])
        
        return self._perturbate(gammars.T), params
        
    
    def EM(self, data, max_iters=250):
        """
        Data: tuple of lists: (y, n)
        """
        # init parameters 1) k-means for latent gammas, 
        #                 2) MLE for pis given gammas, (take proportion)
        #                 3) MLE for alphas and betas given pis and noised gammas values
        E_gammas, params = self._init_gamma_with_kmeans(data)
        init_res = self._M_step(data, E_gammas, params) # init
        params = init_res.x
        print("="*25)
        print("Init params: {}".format(params))
        print("="*25)
        
        neg_log_likelihood = self._get_nnl_fun()        
        losses = [sys.maxsize]
        for ith in range(max_iters):
            
            # E step
            for i in range(self.n_components):
                a, b, pi = params[i], params[i+self.n_components], params[i+2*self.n_components]
                E_gammas[i] = self._E_step(data, a, b, pi)

            # normalize as they are havn't been
            E_gammas = E_gammas / np.sum(E_gammas, axis=0)
            
            # M step
            res = self._M_step(data, E_gammas, params)
            params = res.x
            
            # current NLL loss
            losses.append(neg_log_likelihood(params, data, E_gammas, self.n_components))
            print("="*10, "Iteration {}".format(ith+1), "="*10)
            print("Negative LogLikelihood Loss: {}".format(losses[-1]))
            print("="*25)
            
            improvement = losses[-2] - losses[-1]
            if improvement < self.tor:
                print("Improvement halts, early stop training.")
                break            
            
        self.params = params
        return params

In [114]:
## simulation experiment
# sample gammars
n_samples = 2000
n_trials = 1000
pis = [0.6, 0.4]
alphas, betas = [2, 0.9], [0.1, 5]

gammars = bernoulli.rvs(pis[0], size=n_samples)
n_pos_events = sum(gammars)
n_neg_events = n_samples - n_pos_events

ys_of_type1 = betabinom.rvs(n_trials, alphas[0], betas[0], size=n_pos_events)
ys_of_type2 = betabinom.rvs(n_trials, alphas[1], betas[1], size=n_neg_events)


ys = np.concatenate((ys_of_type1, ys_of_type2))
ns = np.ones(n_samples, dtype=np.int) * n_trials
len(ys), len(ns)

(2000, 2000)

In [115]:
em_mbb = MixtureBetaBinomial(
     n_components=2,
     max_m_step_iter=250,
     tor=1e-20)
params = em_mbb.EM((ys, ns))
print(params)
print(alphas, betas, pis)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in subtract
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in add


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9860.294035431058
            Iterations: 31
            Function evaluations: 283
            Gradient evaluations: 31
Init params: [2.81779529 0.90954636 0.10852537 5.32634319 0.59492329 0.40507671]
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9901.521095932787
            Iterations: 16
            Function evaluations: 160
            Gradient evaluations: 16
========== Iteration 1 ==========
Negative LogLikelihood Loss: 9901.521095932787
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9912.409511748694
            Iterations: 8
            Function evaluations: 88
            Gradient evaluations: 8
========== Iteration 2 ==========
Negative LogLikelihood Loss: 9912.409511748694
Improvement halts, early stop training.
[2.40088949 0.86540789 0.10580962 4.95823576 0.59897059 0.40102941]
[2, 0.9] [0.1, 5] [

1.0